In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")

label_methods = [
    # "atr",
    "trend",
    # "trend_profit",
    # "trend_multi",
    # "clusters",
    # "multi_window",
    # "validated_levels",
    # "zigzag",
    # "mean_rev",
    # "mean_rev_multi",
    # "mean_rev_vol",
    # "filter",
    # "multi_filter",
    # "filter_bidirectional",
    # "filter_one",
    # "trend_one",
    # "filter_flat",
]

base_cfg = dict(
    symbol='XAUUSD', timeframe='H1', direction='both',
    train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    test_start=datetime(2021,6,1),  test_end=datetime(2022,6,1),
    search_type='lgmm', n_trials=1000, n_models=1
)

configs = []
for lm in label_methods:
    cfg = base_cfg.copy()
    cfg["label_method"] = lm
    configs.append(cfg)

# Calcular núcleos por configuración
# total_cores = os.cpu_count()
# cores_per_config = max(1, total_cores // len(configs))

# Actualizar n_jobs en cada configuración
for cfg in configs:
    #cfg["n_jobs"] = cores_per_config
    cfg["tag"] = f"{cfg['symbol']} {cfg['timeframe']} {cfg['direction']} {cfg['label_method']} {cfg['search_type']} {cfg.get('search_subtype', '')}".strip()

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

1
[XAUUSD H1 both atr lgmm] modelo 0 trial 1/1000 ins=0.794412 oos=0.041323 avg=24.89s mem=615.98MB check_constant_features:0.25MB get_train_test_data:184.96MB _apply_labeling:0.00MB fit_final_models:107.59MB _apply_labeling:0.00MB fit_final_models:38.81MB _apply_labeling:0.00MB fit_final_models:37.80MB _apply_labeling:0.00MB fit_final_models:6.98MB _apply_labeling:0.00MB fit_final_models:18.95MB _apply_labeling:0.00MB fit_final_models:6.16MB _evaluate_clusters:197.51MB search_lgmm:385.04MB
1
[XAUUSD H1 both atr lgmm] modelo 0 trial 2/1000 ins=0.794412 oos=0.041323 avg=19.07s mem=646.46MB check_constant_features:0.00MB get_train_test_data:1.00MB _apply_labeling:1.50MB fit_final_models:11.87MB _apply_labeling:0.00MB fit_final_models:11.02MB _apply_labeling:0.00MB fit_final_models:2.86MB _apply_labeling:0.00MB fit_final_models:24.76MB _apply_labeling:0.00MB fit_final_models:8.50MB _apply_labeling:0.00MB fit_final_models:6.43MB _apply_labeling:0.00MB fit_final_models:3.61MB _apply_labelin